In [4]:
import re

In [14]:
## Reading the txt file
## ref : https://www.computerhope.com/issues/ch001721.htm

## Config
input_txt = 'result_Be10-copy.txt'

filenames = {}
filenames["input_file"] = input_txt   ## source data file name must be in /Data/source_data.txt
filenames["input_file_split"] = re.split('-|_|\\.', filenames["input_file"])
nucleus_name = filenames["input_file_split"][1]
filenames["output_csv"] = nucleus_name+"-source.csv"

In [15]:
extracted_data = []
pattern_hf = re.compile("HF INDEX AND ISOSPIN")

with open (filenames["input_file"], 'rt') as myfile:    
    for line in myfile:
        if pattern_hf.search(line) != None:      # If a match is found
            extracted_data.append((line.rstrip('\n')).lstrip('HF INDEX AND ISOSPIN'))

In [16]:
import pandas as pd
obs_twobody_df = pd.DataFrame(columns = ["i1","isoi1","i2","isoi2","f1","isof1","f2","isof2","V"])
obs_twobody_df

for counter, extracted_row in enumerate(extracted_data):
    appending_row = []
    split_string_list = extracted_row.split(" ")
    for split in split_string_list:
        if split =="":
            pass
        else:
            appending_row.append(float(split))
    obs_twobody_df.loc[counter] = appending_row
obs_twobody_df.to_csv(filenames["output_csv"])

## onebody terms

In [18]:
def remove_fermi(energy_list):
    pattern_fermi = re.compile("Fermi")
    for line in energy_list:
        if pattern_fermi.search(line) != None:
            energy_list.remove(line)
    return energy_list

In [24]:
## Extract information for 1body energy level
extracted_data_1b1 = []
pattern_n_block = re.compile("Features for neutrons:")
pattern_p_block = re.compile("Features for protons:")
pattern_energy_start = re.compile("-------------------- Lower bound for Pairing Valence Space -------------------------")
pattern_energy_end   = re.compile("-------------------- Upper bound for Pairing Valence Space -------------------------")
linenum = 0
n_identifier = [] # 1st is neutron block start; 2nd is lowerbound; 3rd is upperbound. Same for proton
p_identifier = []

## the code below assumes neutron's data will come first, then proton's
## also assume the pattern keywords to look for the line are as shown above.

text_whole = []
with open (filenames["input_file"], 'rt') as myfile:
    for line in myfile:
        linenum += 1
        text_whole.append(line)
        ## neutron
        if pattern_n_block.search(line) != None:
            n_identifier.append(linenum)
        if pattern_energy_start.search(line) != None:
            if len(n_identifier) >= 2:
                pass
            else:
                n_identifier.append(linenum)
        if pattern_energy_end.search(line) != None:
            if len(n_identifier) >= 3:
                pass
            else:
                n_identifier.append(linenum)

        ## proton
        if pattern_p_block.search(line) != None:
            p_identifier.append(linenum)
        if linenum in n_identifier:
            pass
        else:
            if pattern_energy_start.search(line) != None:
                p_identifier.append(linenum)
            if pattern_energy_end.search(line) != None:
                p_identifier.append(linenum)

print(n_identifier,p_identifier)

[1011, 1027, 1043] [1054, 1070, 1078]


In [26]:
energy_neu = text_whole[n_identifier[1] : n_identifier[2] -1 : 1]
energy_pro = text_whole[p_identifier[1] : p_identifier[2] -1 : 1]

energy_neu = remove_fermi(energy_neu)
energy_pro = remove_fermi(energy_pro)

obs_onebody_df = pd.DataFrame(columns = ["hf_lvl","iso", "epsilon"])

for counter, line in enumerate(energy_neu):
    dummy_append = []
    dummy_df_row = []
    split_string_list = line.split("  ")
    for split in split_string_list:
        if split =="":
            pass
        else:
            if ("(" in split) or (")" in split):
                pass
            else:
                dummy_append.append(float(split))
    dummy_df_row = [dummy_append[0],dummy_append[2]]
    obs_onebody_df.loc[counter] = dummy_df_row

for counter, line in enumerate(energy_pro):
    dummy_append = []
    dummy_df_row = []
    split_string_list = line.split("  ")
    for split in split_string_list:
        if split =="":
            pass
        else:
            if ("(" in split) or (")" in split):
                pass
            else:
                dummy_append.append(float(split))
    dummy_df_row = [dummy_append[0],dummy_append[2]]
    obs_onebody_df.loc[counter] = dummy_df_row

obs_onebody_df

,lvl,epsilon
0,3.0,-16.052186
1,4.0,-16.052186
2,5.0,-15.614439
3,6.0,-15.614439
4,7.0,-8.278158
5,8.0,-8.278158


987


In [93]:
## to improve next time: 
## connect the gap between this to producing proper matrix elements that is fed directly into the VQE algorithm.